# Global registration with RANSAC
We are going to use open3d (http://www.open3d.org/) to handle  pointclouds and generation of pointclouds

So make sure to call **pip install open3d**


In [3]:
import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

We need to read in our pointclouds.

In [9]:
source = o3d.io.read_point_cloud("ICP/r1.pcd")
target = o3d.io.read_point_cloud("ICP/r3.pcd")

# Used for downsampling.
voxel_size = 0.05

# Show models side by side
draw_registrations(source, target)

### Finding features in pointclouds
When working on point clouds it can be benefitial work on a downsampled version of the point cloud.
you can use [```pointcloudname.voxel_down_sample()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html) where pointcloud is the name of your point cloud object.

We also need to estimate the normals of the pointcloud points using [```pointcloudname.estimate_normals()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html)

And finally find fpfh features or correspondance of the downsampled point clouds.
[```o3d.registration.compute_fpfh_feature()```](http://www.open3d.org/docs/latest/python_api/open3d.registration.compute_fpfh_feature.html#open3d.registration.compute_fpfh_feature)


In [15]:
####
# Downsample and find features here
####
voxel_size = 0.05
radius_feature = voxel_size * 5
# Downsample
source_downsampled = source.voxel_down_sample(voxel_size=voxel_size)
target_downsampled = target.voxel_down_sample(voxel_size=voxel_size)

# Estimating normals of the pointcloud points using
print("Recompute the normal of the downsampled point cloud")
source_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
target_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# o3d.visualization.draw_geometries([source, target])

# Compute features
source_downsampled_fpfh = o3d.registration.compute_fpfh_feature(source_downsampled, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
target_downsampled_fpfh = o3d.registration.compute_fpfh_feature(target_downsampled, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
print(source_downsampled_fpfh.num)

# Code

Recompute the normal of the downsampled point cloud
<bound method PyCapsule.num of registration::Feature class with dimension = 33 and num = 4760
Access its data via data member.>


### Ransac
We will now attempt to use ransac to do a global registration of the two poinclouds.

Using the function [```o3d.registration.registration_ransac_based_on_feature_matching```](http://www.open3d.org/docs/latest/python_api/open3d.registration.registration_ransac_based_on_feature_matching.html#open3d.registration.registration_ransac_based_on_feature_matching) from open3d


Try to find the transformation from r1 to r2.
Attempt with point to point and point to plane
```Python
point_to_point =  o3d.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()
```

When using ransac focus on the arguments below the rest are optional
```Python
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point)
```

In [6]:
####
# Call RANSAC here
####


distance_threshold = voxel_size * 1.5
result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_downsampled, target_downsampled, source_downsampled_fpfh, target_downsampled_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
draw_registrations(source, target, result.transformation, True)

## Exercises
### A)
    Can you get a decent transformation from r1 to r3?
### B)
    with the following checkers can you get better results from RANSAC? Try tweaking the parameters of them Can you make Point to Plane work do not spend too long on this if you cant skip it. (I was not able to get a good fit)

You can also try tweaking the voxel_size
```Python
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.registration.CorrespondenceCheckerBasedOnNormal(0.095)

cecker_list = [c0,c1,c2]

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point
    checkers = checker_list)
```
